In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
import sys
import torch
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset
from tqdm.notebook import trange, tqdm

In [3]:
# Run this cell only if you run this notebook on Google Colab.
from google.colab import drive
drive.mount('/content/drive')

sys.path.append('/content/drive/My Drive/DL_project')

ModuleNotFoundError: No module named 'google.colab'

In [3]:
# Run this cell only if you do NOT run this notebook on Google Colab.

ROOT_DIR = os.path.dirname(os.getcwd())
sys.path.append(ROOT_DIR)

In [4]:
from path import create_dirs
create_dirs()
from path import DATA_TRAIN, OUT_DIR, MODELS_DIR, generate_model_filename, generate_log_filename
from unet import UNet

In [5]:
class ImgDataset(Dataset):
    """Dataset of nuclear fusion time series."""
    def __init__(self, data_dir: str) -> None:
        self.data_dir = data_dir
        self.noisy_imgs1 ,self.noisy_imgs2 = torch.load(data_dir)
        # self.noisy_imgs1 = self.noisy_imgs1[0:100, ...]
        # self.noisy_imgs2 = self.noisy_imgs2[0:100, ...]

    def __getitem__(self, index):
        x = self.noisy_imgs1[index, ...]/255.0
        y = self.noisy_imgs2[index, ...]/255.0
        return x, y

    def __len__(self):
        return self.noisy_imgs1.shape[0]

data = ImgDataset(DATA_TRAIN)
trainNumbers = int(len(data)*0.9)
dataNumbers = [trainNumbers, len(data)-trainNumbers]
trainDataset, testDataset = torch.utils.data.random_split(data, 
                                                          dataNumbers)
trainDataLoader=torch.utils.data.DataLoader(trainDataset, batch_size=16,
                                            shuffle=True)
testDataLoader=torch.utils.data.DataLoader(testDataset,batch_size=16,
                                           shuffle=True)

In [6]:
model = UNet()
optimizer = torch.optim.Adam(model.parameters())
criterion = torch.nn.MSELoss()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model.train()
model.to(device)

cpu


UNet(
  (_block1): Sequential(
    (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (_block2): Sequential(
    (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (_block3): Sequential(
    (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): ConvTranspose2d(48, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
  )
  (_block4): Sequential(
    (0): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(96, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inpl

In [9]:
epochs=15
train_loss = []
test_loss = []

with trange(1, epochs + 1, desc='Training', unit='epoch') as t:

    for epoch in t:
        losses = []

        with tqdm(trainDataLoader, desc=f'Train epoch {epoch}', unit='batch', 
                  leave=False) as t1:
            for x_train ,y_train in t1:
                x_train = x_train.to(device)
                y_train = y_train.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward + backward + optimize
                output = model(x_train)
                loss = criterion(output, y_train)
                loss.backward()
                optimizer.step()
                
                # Append loss
                losses.append(loss)
        
        train_loss.append(sum(losses)/float(len(losses)))

        with torch.no_grad():
            losses = []

            with tqdm(testDataLoader, desc=f'Test epoch {epoch}', unit='batch', 
                  leave=False) as t1:
                for x_test, y_test in t1:

                    x_test = x_test.to(device)
                    y_test = y_test.to(device)

                    output = model(x_test)
                    loss = criterion(output, y_test)
                    losses.append(loss)
            
        test_loss.append(sum(losses)/float(len(losses)))

# save model
filename = generate_model_filename()
print(filename)
weights = model.state_dict()
torch.save(weights,filename)

# save losses
filename = generate_log_filename()
history = dict()
history["train_loss"] = train_loss
history["test_loss"] = test_loss
with open(filename, 'wb') as f:
    pickle.dump(losses, f)


Training:   0%|          | 0/15 [00:00<?, ?epoch/s]

Train epoch 1:   0%|          | 0/2813 [00:00<?, ?batch/s]

AttributeError: 'tqdm_notebook' object has no attribute 'shape'

In [20]:
print(filename)

C:\Users\manon\Desktop\Deep Learning\Projet1\models\log-20220420.pickle
